# Face detection

In [24]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw, ImageFilter, ImageFont
from IPython import display
from collections import namedtuple


Coordinates = namedtuple("Coordinates", ["x0", "x1", "y0", "y1"])

file_name = "video.mp4"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# face detector
mtcnn = MTCNN(keep_all=True, device=device)
video = cv2.VideoCapture(file_name)
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

frames_tracked = []
font = ImageFont.truetype("arial.ttf", 36)


def make_ellipse_mask(size, coordinates, ellipse_blur=10):
    mask = Image.new("L", size, color=0)
    draw = ImageDraw.Draw(mask)
    x0, y0, x1, y1 = coordinates
    ellipse_size = [x0 * 0.95, y0 * 0.95, x1 * 1.05, y1 * 1.05]
    draw.ellipse(ellipse_size, fill=255)
    return mask.filter(ImageFilter.GaussianBlur(radius=ellipse_blur))


while frames_to_read := True:
    frames_to_read, frame = video.read()
    if not frames_to_read:
        break
    frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame_draw = frame.copy()
    frame_array = np.array(frame_draw)
    # scale for faster detections
    scale = 0.5
    frame_array = cv2.resize(
        frame_array, (int(frame_draw.size[0] * scale), int(frame_draw.size[1] * scale))
    )
    draw = ImageDraw.Draw(frame_draw)
    # detect faces
    faces, _ = mtcnn.detect(frame_array)
    if faces is None:
        continue
    draw.text((20, 50), f"Faces: {len(faces)}", color="white", font=font)

    blurred = frame_draw.filter(ImageFilter.GaussianBlur(40))
    for face in faces:
        coordinates = Coordinates(*[x/scale for x in face.tolist()])
        mask_box = make_ellipse_mask(frame_draw.size, coordinates)
        frame_draw.paste(blurred, mask=mask_box)
        draw.rectangle(coordinates, outline="red")
    frames_tracked.append(frame_draw.resize((640, 380), Image.BILINEAR))

dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*"FMP4")
filename_chunks = file_name.split(".")
processed_name = (
    f"{'_'.join([*filename_chunks[:1], 'processed'])}.{filename_chunks[-1]}"
)
video_tracked = cv2.VideoWriter(processed_name, fourcc, fps, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()


OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [3]:
display.Video(processed_name)

AttributeError: 'function' object has no attribute 'Video'